In [ ]:
# Clone into the EMIT Resources Repository to access to helpful visualization tools, ONLY DO THIS ONCE
#!git clone https://github.com/nasa/EMIT-Data-Resources.git
#!cd EMIT-Data-Resources/python/modules
#!cp EMIT-Data-Resources/python/modules/emit_tools.py /home/jupyter/HIR_EMIT/hyperspectral_image_reconstruction_EMIT.ipynb

In [1]:
# Install necessary Python libraries
#!pip install torch --quiet
#!pip uninstall numpy --quiet
!pip install numpy --quiet
!pip install hvplot --quiet
!pip install netCDF4 --quiet
!pip install spectral --quiet
!pip install rasterio --quiet
!pip install rioxarray --quiet
!pip install cartopy geoviews --quiet
!pip install s3fs --quiet

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import torch as nn
import os
import numpy as np
import rioxarray
import xarray as xr
import holoviews as hv
import hvplot.xarray
import netCDF4 as nc
import math
import warnings
import sys
sys.path.append('/home/jupyter/HIR_EMIT/EMIT-Data-Resources/python/modules')

from emit_tools import emit_xarray

warnings.simplefilter('ignore')
hv.extension('bokeh')

In [3]:
class HyperspectralImageProcessor:
    def __init__(self, input_file, output_file):
        self.input_file = input_file
        self.output_file = output_file
        
    def load_dataset(self):
        """
        Load in the hyperspectral image and separate 'sensor_band_parameters' and 'location' in order to access any additional data dimeensions we need
        (i.e. wavelengths and geographic details).
        """
        
        # Initially opening file into xarray
        ds_nc = nc.Dataset(self.input_file)
        ds = xr.open_dataset(self.input_file)
        
        wvl = xr.open_dataset(file_path, group='sensor_band_parameters')
        loc = xr.open_dataset(file_path, group='location')
        
        # Set the group's data as coordinates to the main dataset (using the spatial variables of downtrack and crosstrack)
        ds = ds.assign_coords({'downtrack': ds.downtrack.data,
                       'crosstrack': ds.crosstrack.data,
                       **wvl.variables,
                       **loc.variables})
        
        # Since these datasets are large, we can go ahead and delete objects we won't be using to conserve memory.
        del wvl
        del loc
        
        # Switch dimensions of the dataset from bands to wavelength to provide easier access to our spectral data
        ds = ds.swap_dims({'bands': 'wavelengths'})
        return ds
    
    def orthorectify(self):
        """
        Orthorectification of the granule
        """
        ds_geo = emit_xarray(self.input_file, ortho=True)
        ds_geo.reflectance.data[ds_geo.reflectance.data == -9999] = np.nan  # Mask invalid values
        return ds_geo
    
    def save_chip_to_disk(self, chip, chip_index):
        """
        Save a single chip to a NetCDF file only if it has valid reflectance data (i.e., not all NaNs).

        Parameters:
            chip: The chip to be saved (xarray Dataset or DataArray)
            chip_index: The index of the chip (for naming the output file)
        """
        # Check if the reflectance data has any valid (non-NaN) values
        if chip.reflectance.isnull().any():
            print(f"Chip {chip_index} contains NaN values and will not be saved.")
        else:
            output_file = f"{self.output_file}chip_{chip_index}.nc"
            chip.to_netcdf(output_file)
    
    def split_into_chips(self, ds_oc, chip_size=(64,64), overlap_percent=10):
        """
        Split the orthorectified hyperspectral image into smaller chips (subsets).
        
        Parameters:
            ds_geo: The orthorectified dataset
            chip_size: Tuple (height, width) for the chip size
            overlap: Whether to allow overlap between chips (default is False)
            
        Returns:
            List of chips (subsets of the image data)
        """
        lat_height,lon_width = chip_size
        chips_saved = 0
        
        # Calculate step size based on overlap_percent
        lat_step = int(lat_height * (1 - overlap_percent / 100))
        lon_step = int(lon_width * (1 - overlap_percent / 100))
        
        data_lat, data_lon, num_wavelength = ds_oc.reflectance.shape
        
        # Loop over the granule to extract chips
        for i in range(0, data_lat, lat_step):
            for j in range(0, data_lon, lon_step):
                # Ensure the chip fits within the bounds of the image
                chip = ds_oc.isel(latitude=slice(i, i+lat_height), longitude=slice(j, j+lon_width))
                
                # Save the chip to disk
                chips_saved += 1
                self.save_chip_to_disk(chip, chips_saved)
                
                # Delete the chip from memory to free up space
                del chip
        
        print(f"Saved {chips_saved} chips to disk.")

In [4]:
# State the location of the granule you are currently looking at and the output directory of where you want your chips to be
file_path = "EMIT_DATASET/EMIT_L2A_RFL_001_20241028T045630_2430203_014.nc"
output_path = "chips_1/"
processor = HyperspectralImageProcessor(file_path, output_path)

In [5]:
# Uncomment this cell to load, orthorectify, and split the granule into chips
ds = processor.load_dataset()
ds_oc = processor.orthorectify()
processor.split_into_chips(ds_oc)

Chip 1 contains NaN values and will not be saved.
Chip 2 contains NaN values and will not be saved.
Chip 3 contains NaN values and will not be saved.
Chip 4 contains NaN values and will not be saved.
Chip 5 contains NaN values and will not be saved.
Chip 6 contains NaN values and will not be saved.
Chip 7 contains NaN values and will not be saved.
Chip 8 contains NaN values and will not be saved.
Chip 9 contains NaN values and will not be saved.
Chip 10 contains NaN values and will not be saved.
Chip 11 contains NaN values and will not be saved.
Chip 12 contains NaN values and will not be saved.
Chip 13 contains NaN values and will not be saved.
Chip 14 contains NaN values and will not be saved.
Chip 15 contains NaN values and will not be saved.
Chip 16 contains NaN values and will not be saved.
Chip 17 contains NaN values and will not be saved.
Chip 18 contains NaN values and will not be saved.
Chip 19 contains NaN values and will not be saved.
Chip 20 contains NaN values and will not

In [16]:
class HyperspectralChipProcessor:
    def __init__(self, input_dir, reflec_dir):
        self.input_dir = input_dir
        self.reflec_dir = reflec_dir
        
        if not os.path.exists(self.reflec_dir):
            os.makedirs(self.reflec_dir)

    def process_chip(self, file_path):
        chip = xr.open_dataset(file_path)
        
        reflectance = chip.reflectance.values
        reflectance_filename = os.path.join(self.reflec_dir, os.path.basename(file_path).replace('.nc', '_reflec.npy'))
        np.save(reflectance_filename, reflectance)
        print(f"Saved updated reflectance data for {os.path.basename(file_path)} to {reflectance_filename}")

    def process_all_chips(self):
        chip_files = [f for f in os.listdir(self.input_dir) if f.endswith('.nc')]
        for chip_file in chip_files:
            file_path = os.path.join(self.input_dir, chip_file)
            self.process_chip(file_path)

In [17]:
# State the chip directory and where you want the reflectance numpy arrays to be
reflec_dir = "reflectance_1/"
chip_processor = HyperspectralChipProcessor(output_path, reflec_dir)

In [18]:
# Uncomment this cell to generate the reflectance numpy array
chip_processor.process_all_chips()

Saved updated reflectance data for chip_932.nc to reflectance_1/chip_932_reflec.npy
Saved updated reflectance data for chip_866.nc to reflectance_1/chip_866_reflec.npy
Saved updated reflectance data for chip_1221.nc to reflectance_1/chip_1221_reflec.npy
Saved updated reflectance data for chip_762.nc to reflectance_1/chip_762_reflec.npy
Saved updated reflectance data for chip_698.nc to reflectance_1/chip_698_reflec.npy
Saved updated reflectance data for chip_789.nc to reflectance_1/chip_789_reflec.npy
Saved updated reflectance data for chip_867.nc to reflectance_1/chip_867_reflec.npy
Saved updated reflectance data for chip_607.nc to reflectance_1/chip_607_reflec.npy
Saved updated reflectance data for chip_690.nc to reflectance_1/chip_690_reflec.npy
Saved updated reflectance data for chip_1017.nc to reflectance_1/chip_1017_reflec.npy
Saved updated reflectance data for chip_572.nc to reflectance_1/chip_572_reflec.npy
Saved updated reflectance data for chip_492.nc to reflectance_1/chip_492

In [ ]:
# Uncomment this cell, to visualize the entire granule or a singular chip
#chip_1 = xr.open_dataset('chips/chip_1000.nc')
#chip_1_wavelength = chip_1.sel(wavelengths=385, method='nearest')
#chip_1_wavelength.hvplot.image(cmap='viridis', aspect='equal', frame_width=720).opts(title=f"{chip_1_wavelength.wavelengths.values:.3f} {chip_1_wavelength.wavelengths.units}")
#chip_1.sel(wavelengths=385, method='nearest').hvplot.image(cmap='Viridis', geo=True, tiles='ESRI', alpha=0.8, frame_height=600).opts(
#    title=f"Reflectance at {chip_1_wavelength.wavelengths.values:.3f} {chip_1_wavelength.wavelengths.units} (Orthorectified)"
#)